In [60]:
from config.envs import OPENAI_API_TYPE, OPENAI_BASE_URL, OPENAI_API_KEY, TEMPERATURE, STOP, MAX_TOKENS, TOP_P, FREQUENCY_PENALTY, PRESENCE_PENALTY, N_RESP, TIMEOUT, DB_HOST, DB_PORT, DB_USER, DB_PASSWORD, DB_NAME
import openai
import psycopg2

# Establish a connection to the PostgreSQL server
conn = psycopg2.connect(host=DB_HOST, database=DB_NAME, user=DB_USER, password=DB_PASSWORD, port=DB_PORT)

# setup openai
if OPENAI_API_TYPE == 'azure':
    openai.api_base = OPENAI_BASE_URL
    openai.api_key = OPENAI_API_KEY
    openai.api_type = OPENAI_API_TYPE
    openai.api_version = "2023-07-01-preview"
else:
    openai.api_key = OPENAI_API_KEY

In [61]:
system_prompt_gen = open('config/system_prompt.txt', 'r').read()
input_prompt=input()

In [62]:
def ExecuteSQL(sql) -> str:
        """Function to execute SQL query

        Args:
            sql (str): sql query to execute

        Returns:
            str: output of sql query
        """
        conn = psycopg2.connect(host=DB_HOST, database=DB_NAME, user=DB_USER, password=DB_PASSWORD, port=DB_PORT)

        try:
            cur = conn.cursor()
            # Execute a SQL query
            cur.execute(sql)

            # Fetch the results
            results = cur.fetchall()

            # Close the cursor and connection
            cur.close()
            if not results:
                return "PostgresSQL Query executed Successfully"
            else:
                
                return results

        except Exception as e:
            return f"There is some error in SQL query: {str(e)}"
    

In [63]:
Table_Info=ExecuteSQL("SELECT table_name, column_name, data_type FROM information_schema.columns WHERE table_schema = 'public';")

In [64]:
system_prompt_gen = system_prompt_gen.replace('<table_info>', f"{Table_Info}")

In [65]:
response = openai.ChatCompletion.create(
                engine='DIR_ChatBot',
                messages=[{"role":"system", "content":system_prompt_gen}, {"role":"user", "content":input_prompt}],
                temperature=TEMPERATURE,
                max_tokens=MAX_TOKENS,
                top_p=TOP_P,
                frequency_penalty=FREQUENCY_PENALTY,
                presence_penalty=PRESENCE_PENALTY,
                stop=STOP,
                timeout=TIMEOUT,
                n=N_RESP
            )
            
output_response = response['choices'][0]['message']

In [66]:
output_response

<OpenAIObject at 0x1d8b2992810> JSON: {
  "role": "assistant",
  "content": "I'm sorry, I don't understand what you mean by \"j\". Can you please provide more context or information about what you are looking for?"
}

In [67]:
from tabulate import tabulate

def psql(code: str) -> str:
    """
    Function to execute PostgreSQL code.

    Args:
        code (str): PostgreSQL code to execute

    Returns:
        output (str): Output of PostgreSQL query

    Stream:
        code

    Prefix:
        code: ```sql

    Postfix:
        code: ```

    """


    conn = psycopg2.connect(host=DB_HOST, database=DB_NAME, user=DB_USER, password=DB_PASSWORD, port=DB_PORT)

    try:

        cur = conn.cursor()

        cur.execute(code)

        output = cur.fetchall()

        cur.close()

 

        if not output:

            return {"text":"PostgreSQL Query executed Successfully"}

        else:
            headers = [desc[0] for desc in cur.description]

            rows = [headers] + [list(row) for row in output]

            table = tabulate(rows, headers="firstrow", tablefmt="github")  

            return f"```\n{table}\n```"

       

    except Exception as e:

        return {"text":f"There is some error in SQL query: {str(e)}"}

In [68]:
psql(output_response['content'])

{'text': 'There is some error in SQL query: syntax error at or near "I"\nLINE 1: I\'m sorry, I don\'t understand what you mean by "j". Can you ...\n        ^\n'}